In [1]:
import sys
sys.path.append("../../")

In [2]:
import logging
import math

import torch
import wandb
from torch import nn

import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR, CyclicLR

import numpy as np
import pandas as pd

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import torch

In [4]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor, TQDMProgressBar, StochasticWeightAveraging
from pytorch_lightning.loggers import WandbLogger

import logging

import pytorch_lightning as pl
from torch.utils.data import DataLoader

# from src.PetraRQ.PetraRQDatasets import LanguageModellingDataset

In [5]:
from pytorch_lightning import seed_everything

In [6]:
seed_everything(1, workers=True)

Global seed set to 1


1

In [7]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(name)s -  %(message)s',
    level=logging.INFO
)

In [8]:
dev_ds = "../data/dev/"
test_ds = "../data/test/"
train_ds = "../data/train/"

lm_model_path = "../../models/roberta_lm"

In [9]:
special_tokens = [
    '<url>',
    '<email>',
    '<number>',
    '<date>', 
]

In [10]:
from transformers import RobertaConfig, RobertaTokenizerFast

In [11]:
tokenizer = RobertaTokenizerFast.from_pretrained(lm_model_path, max_len=512, use_fast=True)

In [12]:
tokenizer.add_special_tokens({
    'additional_special_tokens': special_tokens
})

0

In [13]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>',
 'additional_special_tokens': ['<url>', '<email>', '<number>', '<date>']}

In [14]:
data_train = pd.read_csv("../../data/train/processed.tsv", delimiter='\t', header=None, encoding="utf8", quoting=0)
data_dev = pd.read_csv("../../data/dev/processed.tsv", delimiter='\t', header=None, encoding="utf8", quoting=0)
labels_train = pd.read_csv("../../data/train/expected.tsv", delimiter='\t', header=None, encoding="utf8", quoting=0)
labels_dev = pd.read_csv("../../data/dev/expected.tsv", delimiter='\t', header=None, encoding="utf8", quoting=0)

In [15]:
unique_labels_tsv = pd.read_csv("../../data/labels.tsv", delimiter='\t', header=None, encoding="utf8", quoting=0)
unique_labels = unique_labels_tsv[0].tolist()

In [16]:
from transformers import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaEmbeddings

In [17]:
model = RobertaModel.from_pretrained(lm_model_path)

Some weights of the model checkpoint at ../../models/roberta_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../../models/roberta_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
embeds = model.embeddings

In [19]:
t1 = torch.ones(2, 512).long()

In [20]:
model(t1).last_hidden_state

tensor([[[-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         ...,
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089]],

        [[-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         ...,
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089],
         [-1.2255,  0.8372,  1.3041,  ...,  0.1445,  0.3960, -2.3089]]],
       grad_fn=<NativeLayerNormBackward0>)

In [21]:
tokenizer.pad_token_id

5

In [22]:
d_model = 768
train_batch_size = 12
dev_batch_size = 6
shuffle = False
steps = 4000
seq_length = 512
overlapping_part = 256
dropout = 0.1
optimizer = "adam"
lr_min=3e-4
lr_max=3e-3
accumulate_grad_batches=1

In [23]:
import torch.utils.data
from tqdm.auto import tqdm


class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, input_texts, input_labels, unique_labels, tokenizer):
        self.input_texts = input_texts
        self.input_labels = input_labels
        self.unique_labels = unique_labels
        self.tokenizer = tokenizer
        self.label2idx = {}

        # self.bp = BasicProcessor()

        for label in self.unique_labels:
            self.label2idx[label] = len(self.label2idx)

        print(self.label2idx)

#         self.tokenized_texts = []


#         for idx in tqdm(range(len(self.input_texts)), desc='Tokenizing texts'):
#             tokenized = self.tokenizer.encode(self.input_texts.iloc[idx][0])
#             self.tokenized_texts.append(tokenized)

    def __len__(self):
        return len(self.input_texts)

    def labels2tensor(self, labels):
        return set([self.label2idx[label.strip().lower()] for label in labels])

    def tensor2labels(self, tensor):
        labels = []
        for idx, label in enumerate(self.unique_labels):
            if tensor[idx] == 1:
                labels.append(label)
        return labels

    def __getitem__(self, idx):
        tokenized = self.tokenizer.encode(self.input_texts.iloc[idx][0])
        labels = self.labels2tensor(self.input_labels.iloc[idx][0].split(' '))

        # item = {
        #     'input_ids': torch.tensor(tokenized['input_ids']),
        #     'attention_mask': torch.tensor(tokenized['attention_mask']),
        #     'labels': torch.zeros([len(self.label2idx)]).index_fill_(0, torch.tensor(list(labels)), 1)
        # }
        return np.array(tokenized), torch.zeros([len(self.label2idx)]).index_fill_(0, torch.tensor(list(labels)), 1).numpy()


In [24]:
dev_ds = ClassificationDataset(data_dev, labels_dev, unique_labels, tokenizer)
train_ds = ClassificationDataset(data_train, labels_train, unique_labels, tokenizer)

{'health': 0, 'media_informations': 1, 'transportation': 2, 'industry': 3, 'work_and_employment': 4, 'agriculture': 5, 'social_life': 6, 'internal_security': 7, 'economy': 8, 'environment': 9, 'european_union': 10, 'energy': 11, 'foreign_policy': 12, 'education': 13, 'law': 14, 'politics_political_parties': 15, 'sports': 16, 'research_science_and_technology': 17, 'taxes': 18}
{'health': 0, 'media_informations': 1, 'transportation': 2, 'industry': 3, 'work_and_employment': 4, 'agriculture': 5, 'social_life': 6, 'internal_security': 7, 'economy': 8, 'environment': 9, 'european_union': 10, 'energy': 11, 'foreign_policy': 12, 'education': 13, 'law': 14, 'politics_political_parties': 15, 'sports': 16, 'research_science_and_technology': 17, 'taxes': 18}


In [25]:
def coll_fn(batch):
    texts = []
    labels = []
    incremental_indexes = []
    
    pad_token = 5
    
    for (text, label) in batch:
        texts.append(torch.tensor(text).to("cpu"))
        labels.append(label)

    ins = nn.utils.rnn.pad_sequence(texts, batch_first=True, padding_value=pad_token)[:, :5120]
    seq_len = ins.shape[1]
    
    rest = seq_len % seq_length
    # print(seq_len, 256 - rest)
    if rest != 0:
        fill_matrix = torch.zeros((ins.shape[0], seq_length - rest)).fill_(pad_token).long()
        ins = torch.cat((ins, fill_matrix), dim=1)

    return ins.numpy(), np.array(labels), (~(ins == 5)).long().numpy()

In [26]:
train_data_loader = DataLoader(
    train_ds,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    persistent_workers=False,
    collate_fn=coll_fn,
)

In [27]:
dev_data_loader = DataLoader(
    dev_ds,
    batch_size=dev_batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    persistent_workers=False,
    collate_fn=coll_fn
)

In [28]:
for (x, y, att) in dev_data_loader:
    print(x, y, att, x.shape, y.shape, att.shape)
    break

Token indices sequence length is longer than the specified maximum sequence length for this model (3481 > 512). Running this sequence through the model will result in indexing errors


[[    4 15916 12987 ...     5     5     5]
 [    4  3364  3348 ...     5     5     5]
 [    4  3364  3348 ...     5     5     5]
 [    4  3364  3348 ...     5     5     5]
 [    4  3364  3348 ...     5     5     5]
 [    4  3364  3348 ...     5     5     5]] [[0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]] [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]] (6, 3584) (6, 19) (6, 3584)


In [29]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(preds, labels):
    # print(preds.shape, preds)
    preds = (preds >= 0.5).astype(int)#.argmax(-1)

    # print(preds.shape, labels.shape)
    # print(preds, labels)

    # try:
    acc = accuracy_score(preds, labels)
    # except ValueError:

    return {
        'accuracy': acc,
        'f1': f1_score(y_true=labels, y_pred=preds, average='weighted'),
        'precision': precision_score(y_true=labels, y_pred=preds, average='weighted'),
        'recall': recall_score(y_true=labels, y_pred=preds, average='weighted')
    }

In [30]:
class PetraRQ(pl.LightningModule):
    def __init__(
            self,
            d_model,
            embeddings,
            model,
            num_labels,
            seq_length,
            overlapping_part,
            steps=1000,
            lr_min=1e-4,
            lr_max=3e-3,
            optim="adam"
    ):
        super(PetraRQ, self).__init__()
        
        self.d_model = d_model
        self.num_labels = num_labels
        self.seq_length = seq_length
        self.overlapping_part = overlapping_part
        self.steps = steps
        self.lr_min = lr_min
        self.lr_max = lr_max
        self.optim = optim
        self.overlapping_part = overlapping_part
        self.memory_norm = nn.LayerNorm(d_model)
        self.out_norm = nn.LayerNorm(d_model)
        self.sigm = nn.Sigmoid()

        assert (self.optim == 'adam' or self.optim == 'adagrad'), 'Optim must be set to "adam" or "adagrad"'

        self.embeds = embeddings
        self.roberta = model
        
        self.to_logits = nn.Linear(d_model, num_labels)
        
        for param in self.roberta.parameters():
            param.requires_grad = False
            
        for param in self.embeds.parameters():
            param.requires_grad = False


    def forward(self, x_in, attention):
        floating_memory = None
        output_hidden_layers = None
        floating_mems = None
        
        # print('x_in', x_in.shape)
        
        i = 0
        while ((output_hidden_layers is None) or (output_hidden_layers + (x_in.shape[1] % self.overlapping_part) < x_in.shape[1])):
            part = self.seq_length / self.overlapping_part
            if floating_memory is None:
                x_pos = self.embeds(x_in[:, :self.seq_length].to(self.device))
            else:
                toks = x_in[:, int((self.overlapping_part * (i+part-1))):int((self.overlapping_part * (i+part)))].to(self.device)
                # print('from', int((self.overlapping_part * (i+part-1))), 'to', int((self.overlapping_part * (i+part))))
                x_pos = self.embeds(toks)
                x_pos = torch.cat((x[:, self.overlapping_part:, :], x_pos), dim=1)
                
            # print('x_pos', x_pos.shape)
            # if output_hidden_layers is not None:
                # print('dzialanie', output_hidden_layers + (x_in.shape[1] % self.overlapping_part), x_in.shape)
            att_part = attention[:, self.overlapping_part*i:(self.overlapping_part*i) + self.seq_length].to(self.device)
            # print(att_part.shape)
            x = self.roberta(inputs_embeds=x_pos, attention_mask=att_part).last_hidden_state
            
            if floating_memory is None:
                floating_memory = x[:, :self.overlapping_part, :]
                output_hidden_layers = overlapping_part
                # output_hidden_layers = x[:, :self.overlapping_part, :]
                # floating_mems = x_pos[:, :self.overlapping_part, :]
            else:
                add = floating_memory + x[:, :self.overlapping_part, :]
                floating_memory = self.memory_norm(add)
                output_hidden_layers += self.overlapping_part
                # output_hidden_layers = torch.cat((output_hidden_layers, x[:, :self.overlapping_part, :]), dim=1)
                # floating_mems = torch.cat((floating_mems, floating_memory), dim=1)
            del(x_pos)
            i += 1

        out = torch.cat((floating_memory, x[:, self.overlapping_part:, :]), dim=1)

        # print(out.shape)
        # out = out[:, -1, :]
        out = out.sum(dim=1)
        out = self.out_norm(out)
        out = self.to_logits(out)
        out = self.sigm(out)
        # out = self.out_norm(out)
        # out = torch.tanh(out)
        # out = (out >= 0.5).long()
        return out

    def configure_optimizers(self):
        if self.optim == 'adagrad':
            optimizer = torch.optim.Adagrad(
                self.parameters(),
                lr=self.lr_min,
                weight_decay=0.01
            )
        elif self.optim == 'adam':
            optimizer = torch.optim.AdamW(
                self.parameters(),
                lr=self.lr_min,
                weight_decay=0.01,
                betas=(0.9, 0.999)
            )

        lr_scheduler = OneCycleLR(
            optimizer,
            max_lr=self.lr_max,
            total_steps=self.steps,
            cycle_momentum=False,
        )

        return [optimizer], [{'scheduler': lr_scheduler, 'interval': 'step'}]
        # return optimizer

    def training_step(self, batch, batch_idx):
        xs, ys, attention = batch
        x = torch.tensor(xs).to("cpu")
        y = torch.tensor(ys).to(self.device)
        attention = torch.tensor(attention).to("cpu")
        
        x = self.forward(x, attention)        

        loss = F.binary_cross_entropy(x, y)
        
        metrics = compute_metrics(x.detach().cpu().numpy(), y.long().cpu().numpy())
        
        self.log('train/f1', metrics['f1'], prog_bar=True, batch_size=x.shape[0])
        self.log('train/accuracy', metrics['accuracy'], prog_bar=True, batch_size=x.shape[0])
        self.log('train/precision', metrics['precision'], prog_bar=True, batch_size=x.shape[0])
        self.log('train/recall', metrics['recall'], prog_bar=True, batch_size=x.shape[0])

        self.log('train/loss', loss, prog_bar=True, batch_size=x.shape[0])
        return {
            'loss': loss, 
            'train_f1': metrics['f1'],
            'train_acc': metrics['accuracy'],
            'train_precision': metrics['precision'],
            'train_recall': metrics['recall']
        }

    def validation_step(self, batch, batch_idx):
        xs, ys, attention = batch
        x = torch.tensor(xs).to("cpu")
        y = torch.tensor(ys).to(self.device)
        attention = torch.tensor(attention).to("cpu")
        
        x = self.forward(x, attention)        

        loss = F.binary_cross_entropy(x, y)
        
        metrics = compute_metrics(x.cpu().numpy(), y.long().cpu().numpy())
        
        self.log('eval/f1', metrics['f1'], prog_bar=True, batch_size=x.shape[0])
        self.log('eval/accuracy', metrics['accuracy'], prog_bar=True, batch_size=x.shape[0])
        self.log('eval/precision', metrics['precision'], prog_bar=True, batch_size=x.shape[0])
        self.log('eval/recall', metrics['recall'], prog_bar=True, batch_size=x.shape[0])

        self.log('eval/loss', loss, prog_bar=True, batch_size=x.shape[0])
        return {
            'val_loss': loss, 
            'val_f1': metrics['f1'],
            'val_acc': metrics['accuracy'],
            'val_precision': metrics['precision'],
            'val_recall': metrics['recall']
        }


In [31]:
petra = PetraRQ(
    d_model=d_model,
    num_labels=len(unique_labels),
    seq_length=seq_length,
    overlapping_part=overlapping_part,
    steps=steps,
    optim=optimizer,
    embeddings=embeds,
    model=model
)

In [32]:
wandb_logger = WandbLogger(
    project="PetraRQ",
    name="PetraRQ_classifier_recurrent_roberta",
    log_model="all"
)

2022-08-26 19:33:40,532 - ERROR - wandb.jupyter -  Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bmarcin. Use `wandb login --relogin` to force relogin


In [33]:
# wandb_logger.experiment.config['batch_size'] = train_batch_size
# wandb_logger.experiment.config['steps'] = steps
# wandb_logger.experiment.config['d_model'] = d_model
# wandb_logger.experiment.config['num_labels'] = len(unique_labels)
# wandb_logger.experiment.config['seq_length'] = seq_length
# wandb_logger.experiment.config['depth'] = depth
# wandb_logger.experiment.config['heads'] = heads
# wandb_logger.experiment.config['dropout'] = dropout
# wandb_logger.experiment.config['optimizer'] = optimizer
# wandb_logger.experiment.config['duplicate_dataset_ratio'] = duplicate_dataset_ratio

In [34]:
trainer = pl.Trainer(
    devices=1,
    max_steps=steps,
    log_every_n_steps=5,
    accelerator='gpu',
    # accumulate_grad_batches=accumulate_grad_batches,
    val_check_interval=0.05,
    # val_check_interval=300,
    default_root_dir='./PetraRQmodel',
    enable_checkpointing=False,
    callbacks=[
        # ModelCheckpoint(
        #     dirpath='./PetraRQmodel/checkpoints',
        #     save_top_k=3,
        #     monitor='eval/loss',
        #     mode='min',
        #     filename='petrarq-{epoch}-{val_loss:.2f}.ckpt'
        # ),
        EarlyStopping(
            monitor='eval/f1',
            mode='min',
            patience=4,
            check_finite=True,
        ),
        LearningRateMonitor(logging_interval='step'),
        TQDMProgressBar(refresh_rate=1),
        # StochasticWeightAveraging(swa_lrs=1e-3)
    ],
    logger=wandb_logger,
    reload_dataloaders_every_n_epochs=0,
    gradient_clip_val=0.5
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [35]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
# trainer.validate(petra, dev_data_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

In [ ]:
trainer.fit(petra, train_data_loader, dev_data_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | memory_norm | LayerNorm         | 1.5 K 
1 | out_norm    | LayerNorm         | 1.5 K 
2 | sigm        | Sigmoid           | 0     
3 | embeds      | RobertaEmbeddings | 12.7 M
4 | roberta     | RobertaModel      | 98.3 M
5 | to_logits   | Linear            | 14.6 K
--------------------------------------------------
17.7 K    Trainable params
98.3 M    Non-trainable params
98.3 M    Total params
393.391   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]